# General

In this tutorial I will show you how to extract the peaks of openstreetmap files, calculate the routes to different parking spaces or train stations and save the map as a PNG file. Before you can start you have to prepare a few things:
    
    1) Create a folder "maps" in the direction where you execute the Python Script if you want to convert the HTML to PNG.
    2) For a succesful running of the package selenium, you need to download a browser driver. I use chromedriver, but this up to you. For more information see: https://chromedriver.chromium.org/downloads . Be aware that your browser has the same version as the driver you want to download.

The format for the locations is NAME (LON,LAT).

Contribuded by Dr. Stephan Fuchs.



# Get the coordinates of the peaks

Of course you can diretly download the peak data via Python commands. But this would be not the best solution in this case. Here were go to the web side http://overpass-turbo.eu/ and insert on the right side the following code: 

node["natural"="peak"]({{bbox}});out;

This will find all geopoints which are labeled as peak. Now go to the right and select the area where you want to extract the peaks. Finally execute the command by clicking on "Run". Now it may take a few moments until the search finishes. Now click on data and save (via copy/save) the data as a GeoJson-File.



In [ ]:
# setup
!mkdir peak-routing-parking-station
!conda create -n peak-routing-parking-station python=3.9 shapely --yes
!cd peak-routing-parking-station
!pip install openrouteservice folium json Pillow

In [1]:
"""
# Normally you save the peak data in a file. For better presentation of the data it is directly in the notebook.
import json
with open('export.geojson') as json_file:
    data = json.load(json_file)
"""



data={
  "type": "FeatureCollection",
  "generator": "overpass-ide",
  "copyright": "The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.",
  "timestamp": "2021-06-25T15:50:42Z",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "@id": "node/251874697",
        "climbing:grade:saxon:min": "VI",
        "climbing:rock": "sandstone",
        "climbing:summit_log": "yes",
        "climbing:trad": "yes",
        "name": "Herkuleskopf",
        "natural": "peak",
        "note": "check and delete this note if correct; object was imported from http://www.wandern-saechsische-schweiz.de/",
        "sport": "climbing"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          14.0429261,
          50.8422103
        ]
      },
      "id": "node/251874697"
    },
    {
      "type": "Feature",
      "properties": {
        "@id": "node/251874698",
        "climbing:grade:saxon:min": "I",
        "climbing:rock": "sandstone",
        "climbing:summit_log": "yes",
        "climbing:trad": "yes",
        "name": "Herkuleswand",
        "natural": "peak",
        "sport": "climbing"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          14.0434616,
          50.84214
        ]
      },
      "id": "node/251874698"
    },
    {
      "type": "Feature",
      "properties": {
        "@id": "node/251875051",
        "climbing:grade:saxon:min": "III",
        "climbing:rock": "sandstone",
        "climbing:summit_log": "yes",
        "climbing:trad": "yes",
        "name": "Schluchtnadel",
        "natural": "peak",
        "sport": "climbing"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          14.0433542,
          50.8423971
        ]
      },
      "id": "node/251875051"
    }
  ]
}




In [5]:
#Extracting the name and the coordinates of the geoJSON file.

Peak_Names=[]
Coordinates=[]
for i in data["features"]:
    Peak_Names. append(i["properties"]["name"])
    Coordinates.append(i["geometry"]["coordinates"])
    print("Name:",i["properties"]["name"],"--- Coordinates",i["geometry"]["coordinates"])


Name: Herkuleskopf --- Coordinates [14.0429261, 50.8422103]
Name: Herkuleswand --- Coordinates [14.0434616, 50.84214]
Name: Schluchtnadel --- Coordinates [14.0433542, 50.8423971]


The starting points are in general a header in a Excel file. So the header is combined with starting location and coordinates in one string.

In [7]:
Starting_Points=["Parkplatz Zollhaus (50.832243,14.067760)","Parkplatz Ottomühle (50.8391190, 14.0451675)","Parkplatz Ostrov(50.8042060, 14.0466107)"]

# Calculate the routes

In [17]:
import time
import openrouteservice
import re
import folium

#from openpyxl import load_workbook #used for EXCEL Sheet
from PIL import Image

For a succesful running of selenium, you need to download a browser driver. I use chrome, but this up to you. For more information see: https://chromedriver.chromium.org/downloads . Be aware that your browser has the same version as the driver you want to download.

In [38]:
# The style for the plotted routes.
def style_function(color): # To style data of the routes
    return lambda feature: dict(color=color,opacity=0.9,weight=4,)


# Extracts the coordinates of the parking places or trainstations
def search_coordinates(text): #Extract Coordinates from Locationname (12.34,56.78)
    p=re.compile("\d+\.\d+") #REGEX
    koor=p.findall(text)
    Name=text.split("(")[0]
    return (float(koor[1]),float(koor[0])),Name
    

# Calculate the distances between Start and Target. 
def calc_distances(Start,Target):
    coords=(Start,Target)
    client = openrouteservice.Client(key='INSERT-YOUR-API-KEY') # Specify your personal API key
    calc_output = client.directions(coords,profile='foot-walking',geometry= 'true',format_out="geojson")
    distance =  round(calc_output["features"][0]["properties"]["summary"]["distance"])
    duration=   round(calc_output["features"][0]["properties"]["summary"]["duration"]/60.)
    print("Distance(m): ",distance," Duration(min): ",duration)
    return int(distance), int(duration), calc_output 


# Calculate the margin of the map
def calc_map_margin(lon_arr,lat_arr):
    difference=0.0025 # Angle for decreasing size
    lon_min=min(lon_arr)-difference # left/east
    lon_max=max(lon_arr)+difference # right/west
    lat_min=min(lat_arr)-difference # up/north
    lat_max=max(lat_arr)+difference # down/south
    return [(lat_min,lon_min),(lat_max,lon_max)]


In [45]:
# Draws the routes, start and endpoints on the map and saves it as a HTML file.
def show_route(Peak,routes_GPS_koor_1,routes_GPS_koor_2,routes_GPS_koor_3,Start_1,Start_2,Start_3,destination):    
    bbox_margin=calc_map_margin([Start_1[0],Start_2[0],Start_3[0],destination[0]],[Start_1[1],Start_2[1],Start_3[1],destination[1]])
    print("Margin of map: ",bbox_margin)
    folium_base = folium.Map(location=(destination), zoom_start=16) # Create map
    folium_base.fit_bounds(bbox_margin)
    
    #Draw Routes
    folium.GeoJson(routes_GPS_koor_1,name='1 Route',style_function=style_function('blue'))  .add_to(folium_base)
    folium.GeoJson(routes_GPS_koor_2,name='2 Route',style_function=style_function('red'))   .add_to(folium_base)
    folium.GeoJson(routes_GPS_koor_3,name='3 Route',style_function=style_function('orange')).add_to(folium_base)

    #Set Marker for Start/Target
    folium.Marker(list(reversed(Start_1)), popup='Start_1',icon=folium.Icon(color='white',icon_color='blue',  icon='taxi',prefix='fa')).add_to(folium_base)
    folium.Marker(list(reversed(Start_2)), popup='Start_2',icon=folium.Icon(color='white',icon_color='red',   icon='taxi',prefix='fa')).add_to(folium_base)
    folium.Marker(list(reversed(Start_3)), popup='Start_3',icon=folium.Icon(color='white',icon_color='orange',icon='taxi',prefix='fa')).add_to(folium_base)
    
    # Draw Peak
    folium.Marker(list(reversed(destination)),   popup='Target') .add_to(folium_base)

    #folium_base.save("maps/"+Peak+".html") #uncomment if you want save HTML and convert it to PNG
    
    return folium_base

In [46]:
# Opens the browser, loads the HTML and saves the map as PNG.
def HTML_TO_PNG(Peak):
    from selenium import webdriver # used for opening the HTML and save as PNG
    driver = webdriver.Chrome(executable_path="chromedriver") #Path to the Chromedriver
    driver.set_window_size(1000, 800)  # choose a resolution
    driver.get("file:///Absolut/path/maps/"+Peak+".html") #Where to find HTML, has to be the absolute path!!! 
    time.sleep(2) # Waiting time for page loading
    driver.save_screenshot("maps/"+Peak+"_largemap.png")    # Save Screenshot of HTML     
    driver.quit() #close Chrome
    
    #Decrease Image Size (optional)
    image = Image.open("maps/"+Peak+"_largemap.png")
    photo_size=1100
    new_image = image.resize((photo_size, int(photo_size*1354./2000.)))
    new_image.save("maps/"+Peak+"_map.png")

In [47]:


i=0
while i<len(Peak_Names):
    Peak=Peak_Names[i]
    print("---------------------------------------------")
    print("Peak:",Peak)
    print("Peak Coordinates:",Target,"\n")
    #Load GPS coordinates of the parkings spaces
    Coordinates_Start_1,Name_Start_1=search_coordinates(Starting_Points[0])
    Coordinates_Start_2,Name_Start_2=search_coordinates(Starting_Points[1])
    Coordinates_Start_3,Name_Start_3=search_coordinates(Starting_Points[2])
    
    #Read Target/Peak GPS coordinations 
    LAT=str(Coordinates[i][0]) 
    LON=str(Coordinates[i][1])
           
    Target=(float(LAT),float(LON))
    print("Start 1 from: ",Name_Start_1)
    print("Starting Coordinates 1", Coordinates_Start_1)
    distance_1, duration_1,routes_GPS_koor_1=calc_distances(Coordinates_Start_1,Target)
    print("")

    print("Start 2 from: ",Name_Start_2)
    print("Starting Coordinates 2", Coordinates_Start_2)
    distance_2, duration_2,routes_GPS_koor_2=calc_distances(Coordinates_Start_2,Target)
    print("")
    
    print("Start 3 from: ",Name_Start_3)
    print("Starting Coordinates 3", Coordinates_Start_3)
    distance_3, duration_3,routes_GPS_koor_3=calc_distances(Coordinates_Start_3,Target)
    print("")
    
    
    #Calculate Routes
    """
    distance_1, duration_1,routes_GPS_koor_1=calc_distances(Coordinates_Start_1,Target)
    distance_2, duration_2,routes_GPS_koor_2=calc_distances(Coordinates_Start_2,Target)
    distance_3, duration_3,routes_GPS_koor_3=calc_distances(Coordinates_Start_3,Target)
    """
    #Show Route on HTML    
    folium_base=show_route(Peak,routes_GPS_koor_1,routes_GPS_koor_2,routes_GPS_koor_3,Start_1,Start_2,Start_3,Target)

    #Save HTML as PNG
    #HTML_TO_PNG(Peak) # Optional: Save HTML as PNG. It is commented out to enable it for online use.   
                            
    i=i+1



---------------------------------------------
Peak: Herkuleskopf
Peak Coordinates: (14.0433542, 50.8423971) 

Start 1 from:  Parkplatz Zollhaus 
Starting Coordinates 1 (14.06776, 50.832243)
Distance(m):  2840  Duration(min):  34

Start 2 from:  Parkplatz Ottomühle 
Starting Coordinates 2 (14.0451675, 50.839119)
Distance(m):  530  Duration(min):  6

Start 3 from:  Parkplatz Ostrov
Starting Coordinates 3 (14.0466107, 50.804206)
Distance(m):  5531  Duration(min):  66

Margin of map:  [(50.801706, 14.040426100000001), (50.844710299999996, 14.07026)]
---------------------------------------------
Peak: Herkuleswand
Peak Coordinates: (14.0429261, 50.8422103) 

Start 1 from:  Parkplatz Zollhaus 
Starting Coordinates 1 (14.06776, 50.832243)
Distance(m):  2807  Duration(min):  34

Start 2 from:  Parkplatz Ottomühle 
Starting Coordinates 2 (14.0451675, 50.839119)
Distance(m):  497  Duration(min):  6

Start 3 from:  Parkplatz Ostrov
Starting Coordinates 3 (14.0466107, 50.804206)
Distance(m):  5498

In [48]:
# Show the routes on the map
folium_base